In [ ]:
# This page is for provisioning an ESP32 GPS device for the wingtip or base-station.  
# This interfaces with the Hanglog3 android app to record the data, which the 
# scripts in this repository can interact with to enable RTK technology


In [20]:
#### esptool.py --port /dev/ttyUSB1 erase_flash
%esptool erase

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.12-606-g07f181a21 on 2020-07-01; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f80107bfc10, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Executing:
  esptool.py --port /dev/ttyUSB0 erase_flash

esptool.py v2.8
Serial port /dev/ttyUSB0
Connecting.......
[Press the PRG button now if required]
Detecting chip type... ESP32
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core, 240MHz, VRef calibration in efuse, Coding Scheme None
Crystal is 40MHz
MAC: 3c:71:bf:ff:67:7c
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 9.2s
Hard resetting via RTS pin...


In [21]:
#### esptool.py --port /dev/ttyUSB0 --chip esp32 write_flash -z 0x1000 /home/julian/executables/micropythonbins/esp32-idf4-20200124-v1.12-87-g96716b46e.bin
%esptool esp32 /home/julian/executables/micropythonbins/esp32-idf4-20200701-unstable-v1.12-606-g07f181a21.bin

Executing:
  esptool.py --port /dev/ttyUSB0 --chip esp32 write_flash -z 0x1000 /home/julian/executables/micropythonbins/esp32-idf4-20200701-unstable-v1.12-606-g07f181a21.bin

esptool.py v2.8
Serial port /dev/ttyUSB0
Connecting....
[Press the PRG button now if required]
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core, 240MHz, VRef calibration in efuse, Coding Scheme None
Crystal is 40MHz
MAC: 3c:71:bf:ff:67:7c
Uploading stub...
Running stub...
Stub running...
Configuring flash size...
Auto-detected Flash size: 4MB
Compressed 1448848 bytes to 924753...
Wrote 1448848 bytes (924753 compressed) at 0x00001000 in 82.2 seconds (effective 141.0 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


In [22]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [23]:
help("modules")

__main__          gc                uasyncio/stream   upip_utarfile
_boot             inisetup          ubinascii         upysh
_onewire          machine           ubluetooth        urandom
_thread           math              ucollections      ure
_uasyncio         micropython       ucryptolib        urequests
_webrepl          neopixel          uctypes           uselect
apa106            network           uerrno            usocket
btree             ntptime           uhashlib          ussl
builtins          onewire           uhashlib          ustruct
cmath             sys               uheapq            utime
dht               uarray            uio               utimeq
ds18x20           uasyncio/__init__ ujson             uwebsocket
esp               uasyncio/core     umqtt/robust      uzlib
esp32             uasyncio/event    umqtt/simple      webrepl
flashbdev         uasyncio/funcs    uos               webrepl_setup
framebuf          uasyncio/lock     upip              websocket_hel

In [624]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [634]:
%sendtofile config.txt

deviceletter   A
pinled         2
pingled        23
pinbatteryadc  34,6800,2200,3.8
connection1    BV6000,beckaaaa,192.168.43.1,9042
connection0    JulianS9,beckaaaa,192.168.43.1,9042

Sent 6 lines (183 bytes) to config.txt.


In [679]:
%sendtofile config.txt

deviceletter   B
pinled         2
pinbatteryadc  34,6800,2200,3.8
connection1    BV6000,beckaaaa,192.168.43.1,9042
connection0    JulianS9,beckaaaa,192.168.43.1,9042

Sent 5 lines (165 bytes) to config.txt.


In [24]:
%sendtofile config.txt

deviceletter   C
pinled         2
pinbatteryadc  34,6800,2200,3.6
connection1    BV6000,beckaaaa,192.168.43.1,9042
connection0    JulianS9,beckaaaa,192.168.43.1,9042
sdcard         5,18,23,19,/sd
wifi2sdtimeout 10

Sent 7 lines (213 bytes) to config.txt.


In [31]:
%sendtofile --source sdcard.py

Sent 281 lines (8634 bytes) to sdcard.py.


In [38]:
%sendtofile --source asyncutils.py

Sent 163 lines (5648 bytes) to asyncutils.py.


In [30]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


Sent 167 lines (5638 bytes) to asyncutils.py.


In [37]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [33]:
import os
os.remove("main.py")

In [34]:
import os
print(os.listdir())

['asyncutils.py', 'boot.py', 'config.txt']


In [39]:
%sendtofile main.py

import uasyncio as asyncio
import uasyncio.stream
from asyncutils import fconfig, wificonnect, initUBX, mountsd, \
                       streamUBX, encodeUBX, pinled, flashseq, batteryvoltage
import socket, os

deviceletter = fconfig["deviceletter"]

fdatebasedname = None
sddir = None
sdfileout = None
nbytesout = 0

androidipnumber, portnumber = None, None
socketstream = None
socketstreamcmd = None

battvoltmsg = None
async def ledflashingtask():
    global battvoltmsg
    lnbytesout = 0
    n = 0
    while True:
        battvolt = batteryvoltage()
        if battvolt != 0 and (n%10) == 0:
            battvoltmsg = encodeUBX(0x21, 0x04, ("battvolt=%4.2fv" % battvolt).encode())  # UBX-LOG-STRING

        battfac = max(0.1, battvolt - 6)/4
        battms = int(battfac*800)
        if sdfileout is None and socketstream is None:
            if fdatebasedname is None:
                await flashseq((500, 50, 100, 50, 200, battms))
            else:
                await flashseq((500, 50, 200, battms))
        
        elif nbytesout - lnbytesout > 40:
            lnbytesout = nbytesout
            if sdfileout is not None:
                await flashseq((1500, 100, 200, 100, 200, battms, 300, 100, 200, 100))
            else:
                await flashseq((1500, 100, 200, battms, 300, 100, ))
                await flashseq((1500, 100, 200, battms, 300, 100, ))
            
        else:
            await flashseq((400, 1000, 200, battms))
        n += 1


        
socketreadlinestack = [ ]
async def socketstreamcmdtask():
    global socketstreamcmd, socketstream, battvoltmsg, sddir
    while len(socketreadlinestack) == 0 or socketreadlinestack[0] != "-SDMODE":
        if fdatebasedname is None and battvoltmsg is not None and socketstream is not None:
            try:
                await socketstream.awrite(battvoltmsg)
            except OSError as e:
                print("OSError writing battvoltmsg")
            battvoltmsg = None 
        await asyncio.sleep_ms(100)
        
    if sddir is None:
        sddir = mountsd()
    socketstreamcmd, socketstream = socketstream, None

    while True:
        try:
            if len(socketreadlinestack) == 3:
                rld, rl = socketreadlinestack.pop(), socketreadlinestack.pop()
                if rl == "-DRL":
                    print("listing", rld)
                    await socketstreamcmd.awrite(("List '%s'\n" % rld).encode())
                    for l in os.listdir(rld):
                        nbytes = os.stat("%s/%s"%(rld,l))[6]
                        await socketstreamcmd.awrite(("%s %d\n" % (l, nbytes)).encode())
                    await socketstreamcmd.awrite((".\n").encode())
                elif rl == "-DRR":
                    print("reading", rld)
                    print(os.stat(rld))
                    nbytes = os.stat(rld)[6]
                    await socketstreamcmd.awrite(("%d\n" % nbytes).encode())
                    fin = open(rld, "rb")
                    while True:
                        d = fin.read(500)
                        if not d:
                            break
                        await socketstreamcmd.awrite(d)
                        if len(socketreadlinestack) >= 2 and socketreadlinestack[1] == "-SDMODE":
                            break
                elif rl == "-DRE":
                    print("erasing", rld)
                    try:
                        os.remove(rld)
                        msg = "Removed '%s'\n"%rld
                    except OSError:
                        msg = "Failed\n"
                    await socketstreamcmd.awrite(msg.encode())
            await asyncio.sleep_ms(100)
        except OSError as e:
            print("socketstream3 OSError", e)
            socketstream = None
            await asyncio.sleep_ms(2000)
        
async def manageconnectiontask():
    global socketstream, sdfileout, socketstreamcmd, sddir
    global androidipnumber, portnumber
    # this won't return with None unless there is an SDCard option
    androidipnumber, portnumber = await wificonnect()  
    
    if androidipnumber is not None:
        while True:
            try:
                if socketstream is None:
                    ss = socket.socket()
                    ss.settimeout(1)
                    print("new socket connection", ss)
                    ss.connect(socket.getaddrinfo(androidipnumber, portnumber)[0][-1])
                    lsocketstream = uasyncio.stream.Stream(ss)
                    print(await lsocketstream.readline())
                    await lsocketstream.awrite(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))
                    if socketstreamcmd is None:
                        socketstream = lsocketstream
                    else:
                        socketstreamcmd = lsocketstream  # drop back into cmd stream on reconnect
                
                while (socketstream is not None) or (socketstreamcmd is not None):
                    print("await socketstream.readline")
                    rl = (await (socketstream or socketstreamcmd).readline()).decode().strip()
                    print("socketstream line", [rl])
                    while len(socketreadlinestack) > 2:
                        socketreadlinestack.pop(0)
                    socketreadlinestack.append(rl)
                    
            except OSError as e:
                print("socketstream2 OSError", e)
                socketstream = None  # cmdstream stays not none as a placeholder
                await asyncio.sleep_ms(2000)
                
    # no wifi found, and timeout set, drop into SD card plotting 
    else:
        while fdatebasedname is None:
            print("waiting for fdatebasedname")
            await asyncio.sleep_ms(1000)
        sddir = mountsd()
        if sddir is not None:
            sdfilename = "%s/%s%s.ubx" % (sddir, fdatebasedname, deviceletter)
            print("Opening sdfile", sdfilename)
            sdfileout = open(sdfilename, "wb")
            for i in range(20):
                await flashseq((100, 50))
        else:
            loop.create_task(manageconnectiontask())
        
async def streamgpstask():
    global fdatebasedname, socketstream, nbytesout, battvoltmsg
    fdatebasedname = await initUBX()
    while True:
        fdata = await streamUBX.read(-1)
        if socketstream is not None:
            try:
                if battvoltmsg is not None and fdata[:4] == b'\xb5\x62\x01\x30':
                    print(battvoltmsg)
                    await socketstream.awrite(battvoltmsg)
                await socketstream.awrite(fdata)
            except OSError as e:
                print("socketstream1 OSError", e)
                socketstream = None  # should kick off connecting again
                
        if sdfileout is not None:
            try:
                if battvoltmsg is not None and fdata[:4] == b'\xb5\x62\x01\x30':
                    sdfileout.write(battvoltmsg)
                sdfileout.write(fdata)
                sdfileout.flush()
            except OSError as e:
                print("SDFile OSError", e)

        if battvoltmsg is not None and fdata[:4] == b'\xb5\x62\x01\x30':
            battvoltmsg = None

        if sdfileout is not None or socketstream is not None:
            np, nbytesout = nbytesout, nbytesout+len(fdata)
            if np//100000 != nbytesout//100000:
                print("bytes:", nbytesout)

loop = asyncio.get_event_loop()
loop.create_task(manageconnectiontask())
loop.create_task(streamgpstask())
loop.create_task(ledflashingtask())
loop.create_task(socketstreamcmdtask())
loop.run_forever()

    

Sent 193 lines (7674 bytes) to main.py.


In [438]:
%ls sd

Listing directory 'sd'.
             sd/OOH/
             sd/System Volume Information/
        0    sd/hdata-2020-07-06_17-18-26C.ubx
   133531    sd/hdata-2020-07-06_17-21-04C.ubx
    28170    sd/hdata-2020-07-06_17-24-25C.ubx
        0    sd/test.txt


In [259]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [576]:
print(b'\xb5b\x010\x04\x01\x10\x1d\xf1\x18'[5])

1


In [591]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 158, in <module>
  File "uasyncio/core.py", line 1, in run_forever
  File "uasyncio/core.py", line 1, in run_until_complete
  File "main.py", line 47, in manageconnectiontask
  File "asyncutils.py", line 81, in wificonnect
KeyboardInterrupt: 
MicroPython v1.12-606-g07f181a21 on 2020-07-01; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [593]:
from asyncutils import mountsd
print(mountsd())

/sd


In [595]:
%ls sd


Listing directory 'sd'.
             sd/OOH/
             sd/System Volume Information/
  1474115    sd/hdata-2020-07-07_20-43-28C.ubx
  3397565    sd/hdata-2020-07-10_09-43-54C.ubx


In [596]:
%fetchfile --binary sd/hdata-2020-07-10_09-43-54C.ubx

Fetched 3397565=3397565 bytes from sd/hdata-2020-07-10_09-43-54C.ubx.
Saving file to 'hdata-2020-07-10_09-43-54C.ubx'